# Parsing

This script parses raw texts into a spreadsheet.

In [21]:
import os
import re
import csv
from operator import itemgetter
from itertools import groupby

## Some helpful functions

In [22]:
def mergeLines(l):
    '''
    This function takes in a list of lines and merge broken paragraph lines 
    (merge all lines if they don't start with a number)
    '''
    i = 0
    while i < len(l):
        if not l[i][0].isdigit():
            l[i-1:i+1] = [' '.join(l[i-1:i+1])]
        else:
            i = i+1
    return(l)

In [23]:
# function to find main paragraphs numbers in each upr
def mainParagraphs(upr):
    '''
    This function takes in a upr and returns the main paragraph numbers in the 'recommendations' section.
    There are usually 2-4 main paragraphs. Sometimes I refer to these main paragraph sections as "chunks".
    '''
    firstParagraph = upr['text'][0].partition(" ")[0]
    if '.' in firstParagraph:
            firstParagraph = firstParagraph.replace(".","")
    mainParagraphs = []

    for line in upr['text']:
        paragraph = line.partition(" ")[0]
        if paragraph[-1] == '.':
            paragraph = paragraph[:-1]
                
        mainParagraphs.append(float(paragraph))
        
    # make a list of the main paragraph numbers
    mainParagraphs = set([int(n) for n in mainParagraphs if int(n)>= int(firstParagraph)])
    return mainParagraphs   

## Read in Data

This for loop will reach in each UPR in a certain date (e.g. 2013) and parse it into a dictionary with country, year, and text keys. The *text* value will be a list of the main paragraphs in the recommendations section of the UPR

In [24]:
l=[]
dir = '../Data/raw-data/txts'
for file_name in os.listdir(dir):
    broken = []
    if file_name.endswith(".txt") and 'haiti2011' in file_name:
        print 'processing ' + file_name + '...'
        try:
            dic = {}
            dic['country'] = file_name[:-8]
            dic['year'] = file_name[-8:-4]
            f = open(dir + '/' + file_name,'rU')
            x = f.read() # read in text
            f.close
            x = x.split('\n') # make a list
            x = filter(None, x) # get rid of empty string items       
             
            # take only the conclusions and/or recommendations section
            ConclusionsStart = x.index([s for s in x if "conclusions and/or recommendations" in s.lower()][1]) #startin from bottom
            ConclusionsEnd = x.index([s for s in x if "conclusions and/or recommendations" in s.lower()][2]) # the last one is the disclaimer
            x = x[ConclusionsStart+1:ConclusionsEnd+1] 
            
            # get rid of the weird lines
            x = [s for s in x if '**' not in s]
            x = [s for s in x if 'recommendations have not been edited.' not in s]
            x = [s for s in x if 'recommendations will not be edited.' not in s]
            x = [s.replace('\xd2','') for s in x]
            x = [s.replace('\t','') for s in x]
            x = [s.lstrip(" ") for s in x]
            
            # get rid of footnotes:
            for s in x:
                if s.rstrip()[-1].isdigit():
                    x[x.index(s)] = s.rstrip()[:-1]
            for s in x:
                if s.rstrip()[-1].isdigit():
                    x[x.index(s)] = s.rstrip()[:-1]

            # merge lines so that each line is its own paragraph, starting with a paragraph number
            x = mergeLines(x)
                       
            # get rid of that disclaimer paragraph
            x = [s for s in x if 'endorsed by the working group' not in s.lower()]
            
            # make sure everything is a string
            dic['text'] = [str(s) for s in x]
            
            # append to list
            l.append(dic)
              
        except Exception,e:
            broken.append(file_name +str(e)) 

processing haiti2011.txt...


In [25]:
# check if anything broke
broken

[]

## If the document does not follow the 67.1, 67.2 format, fix it:

In [26]:
def getnumber(line):
            try:
                number = int(line.partition(".")[0])
            except:
                number= int(line.partition(" ")[0])
            return(number)

In [27]:
for upr in range(len(l)):
    x = l[upr]['text'] 
    firstParagraph = x[0].partition(".")[0]
    secondParagraph = x[1].partition(".")[0]

    if firstParagraph == secondParagraph:
        pass

    else:
        try:
            # make a list of paragraph numbers as they appear in document
            numbers = []
            for line in x:
                numbers.append(getnumber(line))   
        
            # group that list by sequential numbers
            grouped = []
            for k, g in groupby(enumerate(numbers), lambda (i,x):i-x):
                grouped.append(map(itemgetter(1), g))
            
            # tack on main paragraph number to subparagraphs
            groupedNumbers = []
            for list in grouped:
                if list[0] != 1:
                    n = list[-1]
                    groupedNumbers.append([str(i) for i in list])
                if list[0] == 1:
                    groupedNumbers.append([str(n)+'.'+str(number) for number in list])
            
            # make the list presentable
            groupedCollapsed = [item for sublist in groupedNumbers for item in sublist]
        
            # make new statements
            numberedStatements = []
            for i in range(len(x)):
                text = x[i]
                woParagraph = text[len(str(numbers[i]))+1:]
                withnewParagraph = groupedCollapsed[i] + ' ' + woParagraph
                numberedStatements.append(withnewParagraph)
    
            l[upr]['text'] = numberedStatements
        
        except Exception,e:
            print str(e)
            broken.append(l[upr]['country'] + l[upr]['year']+str(e))

In [28]:
# take a look at the second rec of the first 10 uprs
for x in l:
    print x['text'][1]

88.1 Examine the possibility to accede to the International Covenant on Economic, Social and Cultural Rights (Algeria); 


In [29]:
def parse2014(document):
    '''
    This function passes a UPR and parse the 'text' value into well structured format, 
    i.e. a nested (array-like) dictionary of 2-4 main paragraphs, with each main paragraph containing 
    the text of the paragraph, the decision it represents (consider, accept, reject, unknown), 
    and a list of specific recommendations or subparagraphs ('items') under it.
    
    For example, in one upr the returned 'paragraphs' value will contain something like:
    {128: {'decision': 'consider', 'text': string-of-paragraph, 'items': [rec 1, rec 2, ...]}
    {129: {'decision': 'accept', 'text': string-of-paragraph, 'items': [rec 1, rec 2, ...]}
    {130: {'decision': 'reject', 'text': string-of-paragraph, 'items': [rec 1, rec 2, ...]}
    '''
    upr = document
    
    # assign main paragraphs to upr dictionary
    upr['mainP'] = mainParagraphs(upr)
    
    # separate upr out into chunks by mainP, i.e. the support, reject, consider chunks 
    # by identifying subparagraphs that start with the main paragraph numbers.
    upr['chunks'] = {}
    for n in upr['mainP']: # i.e. Paragraphs 123, 124, 125.
        statements = [s for s in upr['text'] if s.startswith(str(n))] # find a list of lines in each main chunk 
        upr['chunks'][n] = statements # assign each chunk
    del upr['text']
    
    # parse each chunk into paragraph texts and subitems, assign to 'paragraphs' key 
    upr['paragraphs'] = {}
    for key in upr['chunks'].keys():
        dic = {}
        dic['text'] = upr['chunks'][key][0] # the main text is always the first paragraph, i.e. the first line.
        if len(upr['chunks'][key]) > 1: # find subitems. (everything after 1st line)
           dic['items'] = [s for s in upr['chunks'][key][1:]]
        else:
            dic['items'] = None # some paragraphs have no subitems.
        upr['paragraphs'][key] = dic # assign
    del upr['chunks']
    
    # assign decision:
    for x in upr['paragraphs'].keys():
        text = upr['paragraphs'][x]['text'] 
        decision = ''
        if 'implemented' in text or 'process of implementation' in text:
            decision = 'implemented'
        elif 'will be examined' in text or 'will examine' in text or "further examined" in text or "Responses to the following recommendations will be provided" in text or "will be included in the outcome report" in text or "will be provided in due course" in text or "course of the discussion" in text:
            decision = 'consider'
        elif 'not enjoy the support' in text or 'reject' in text or 'cannot be accepted' in text:
            decision = 'reject'
        elif 'support' in text and 'did not enjoy the support' not in text:
            decision = 'support'
        elif 'have been noted by' in text or 'were noted by' in text:
            decision = 'noted'
        elif 'do not reflect the current situation' in text:
            decision = 'reject'
        else:
            decision = 'unknown'
        upr['paragraphs'][x]['decision'] = decision
    
    # break down multiple paragraph items
    for key in upr['paragraphs'].keys():
        l = []
        if upr['paragraphs'][key]['items'] is not None:
            for line in upr['paragraphs'][key]['items']:
                n = line.partition(" ")[0] # main par. (chunk) number
                d = ');' # delimiter 
                s = [e+d for e in line.split(d) if e != ""] # split by delimiter, but keep the delimiter.
                s = [e for e in s if e.strip()!=');'] # remove lines containing just the delimiter
                f = [s[0]] # start a new list
                for item in s[1:]:
                    f.append(str(n)+'.'+str(s.index(item))+item) # add items to new list, looking as it should
                l.append(f) # add newly parsed list to main list of UPR recommendations
            l = [i for subitem in l for i in subitem] # flatten list
            upr['paragraphs'][key]['items'] = l # assign
        
    return(upr)

In [30]:
# apply function to the uprs.
newlist = []
for doc in l:
    try:
        newlist.append(parse2014(doc))
    except Exception,e:
        print str(l.index(doc)) + str(e)

In [31]:
# check it out
newlist[-1]

{'country': 'haiti',
 'mainP': {88},
 'paragraphs': {88: {'decision': 'consider',
   'items': ['88.1 Examine the possibility to accede to the International Covenant on Economic, Social and Cultural Rights (Algeria);',
    '88.2 Consider ratifying the ICESCR and continue to strengthen institutions of democratic governance (South Africa);',
    '88.3 Further ratify the International Covenant on Economic, Social and Cultural Rights and the Convention against Torture (Ghana);',
    '88.4 Strengthen its judicial framework by acceding to the International Covenant on Economic, Social and Cultural Rights, to the two Optional Protocols to the International Covenant on the Political and Civilian Rights, to the Convention on Torture and Other Cruel, Inhuman or Degrading Treatments or Punishments and to the International Convention for the Protection of All Persons from Enforced Disappearances (France);',
    '88.5 Ratify the International Covenant on Economic, Social and Cultural Rights, the Con

## Testing Testing 123

In [32]:
# Take a look at the main paragraphs
for x in newlist:
    for paragraph in x['paragraphs']:
        print x['paragraphs'][paragraph]['decision']
        print x['paragraphs'][paragraph]['text'] + ('\n')

### TODO ####
# export this data somehow

consider
88. The following recommendations will be examined by Haiti which will provide responses in due time, but no later than the nineteenth session of the Human Rights Council in March 2012: 



In [33]:
# which decisions are 'unknown'?
for x in newlist:
    for paragraph in x['paragraphs']:
        if x['paragraphs'][paragraph]['decision'] == 'unknown':
            print x['country'] + x['year'] + ('\n')
            print x['paragraphs'][paragraph]['text'] + ('\n')

In [34]:
# write paragraphs with no subitems

with open('specialpars.txt', 'a') as f:
    for upr in newlist:
        for paragraph in upr['paragraphs']:
            if upr['paragraphs'][paragraph]['items'] is None:
                f.write(upr['country'] + upr['year'] + ('\n'))
                f.write(upr['paragraphs'][paragraph]['text'] + ('\n\n'))


In [35]:
# make dictionaries for each individual recommendation item, containing the text, to country, from country, and decision
for upr in newlist:
    l = []
    for key in upr['paragraphs'].keys():
        if upr['paragraphs'][key]['items'] is not None:
            for item in upr['paragraphs'][key]['items']:
                dic = {}
                dic['to'] = upr['country']
                dic['year'] = upr['year']
                dic['decision'] = upr['paragraphs'][key]['decision']
                dic['from'] = item.split('(')[-1].split(')')[0]
                dic['text'] = item
                l.append(dic)
    upr['recs'] = l       

In [36]:
len(newlist[-1]['recs'])

136

In [37]:
# cat recs into a list
recslist = []
for doc in newlist:
    for rec in doc['recs']:
        recslist.append(rec)
len(recslist)

136

## write csv

In [18]:
#writing column headings
keys = recslist[0].keys()
keys

['text', 'to', 'decision', 'from', 'year']

In [38]:
#writing the rest
with open('../Data/year-csvs/haiti2011.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(recslist)